<a href="https://colab.research.google.com/github/audreyemmely/ia-generativa/blob/main/RoBERTa_ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TRAINING

In [ ]:
!pip install transformers datasets huggingface_hub tensorboard
!pip install accelerate -U
!sudo apt-get install git-lfs --yes

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [3]:
notebook_login()

In [4]:
model_id = "roberta-base"
dataset_id = "told-br"
repository_id = "audreyvasconcelos/iag-class-ptbr"

In [5]:
# Load dataset
dataset = load_dataset(dataset_id)

train_dataset = dataset['train']
test_dataset = dataset["test"].shard(num_shards=2, index=0)

val_dataset = dataset['validation'].shard(num_shards=2, index=1)

tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

In [6]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

In [7]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [9]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [8]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16800
})

In [10]:
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

id2label = {i: label for i, label in enumerate(class_names)}

config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

number of labels: 2
the labels: ['not-toxic', 'toxic']


In [11]:
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

/content/audreyvasconcelos/iag-class-ptbr is already a clone of https://huggingface.co/audreyvasconcelos/iag-class-ptbr. Make sure you pull the latest changes with `repo.git_pull()`.


In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.593600,0.540572
2,0.485800,0.539361
3,0.562200,0.514722
4,0.453800,0.531020
5,0.430500,0.579864


TrainOutput(global_step=10500, training_loss=0.47558720518293834, metrics={'train_runtime': 4211.6576, 'train_samples_per_second': 19.945, 'train_steps_per_second': 2.493, 'total_flos': 1.105066432512e+16, 'train_loss': 0.47558720518293834, 'epoch': 5.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.5147220492362976,
 'eval_runtime': 11.5151,
 'eval_samples_per_second': 91.185,
 'eval_steps_per_second': 11.463,
 'epoch': 5.0}

In [17]:
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()

trainer.push_to_hub()

Upload file logs/events.out.tfevents.1692617473.1e5e9e983fa3.1454.1:   0%|          | 1.00/167k [00:00<?, ?B/s…

Upload file logs/events.out.tfevents.1692621998.1e5e9e983fa3.1454.2:   0%|          | 1.00/359 [00:00<?, ?B/s]

To https://huggingface.co/audreyvasconcelos/iag-class-ptbr
   9d0ad27..7e28257  main -> main

   9d0ad27..7e28257  main -> main



'https://huggingface.co/audreyvasconcelos/iag-class-ptbr/commit/7e2825786570c964dc127b6104089b729e462419'

## TESTING

In [18]:
!pip install transformers xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 2.4 MB/s eta 0:00:00


In [38]:
from transformers import pipeline
import pandas as pd

repository_id = "audreyvasconcelos/iag-class-ptbr"
classifier = pipeline('text-classification', repository_id)

data_pairs = [
    ("Eu nunca vi alguém tão incompetente em toda minha vida. Você é uma vergonha para a empresa.", "toxic"),
    ("Deveriam ter escolhido uma modelo menos gordinha, ela parece que não está passando fome!", "toxic"),
    ("Deveriam ter escolhido uma modelo menos gorda, ela parece que não está passando fome!", "toxic"),
    ("Seu namorado é muito feinho, você não tem vergonha de namorar alguém igual a ele?", "toxic"),
    ("Seu namorado é muito feio, você não tem vergonha de namorar alguém igual a ele?", "toxic"),
    ("Seu namorado é muito ridículo, você não tem vergonha de namorar um calvo igual a ele?", "toxic"),
    ("Parabéns pelo seu trabalho árduo e dedicação. Seu esforço realmente faz a diferença.", "not-toxic"),
    ("É inspirador ver o quanto você se esforça para alcançar seus objetivos. Continue assim!", "not-toxic"),
    ("Sua capacidade de resolver problemas de forma criativa e eficaz é impressionante. Continue brilhando!", "not-toxic"),
    ("Meus amigos são pessoas muito idiotas por mais que eu tente ser legal com eles, sempre são escrotos comigo.", "toxic"),
    ("Não aguento mais lidar com suas reclamações constantes e negatividade. Você torna tudo mais difícil.", "toxic"),
    ("Seu apoio e gentileza não passam despercebidos. Você é uma pessoa incrível e generosa.", "not-toxic"),
    ("Você sempre demonstra uma compreensão profunda das situações. Sua perspicácia é admirável.", "not-toxic"),
    ("Você é um merda muito burro", "toxic"),
    ("Você é muito burro e não entende nada.", "toxic"),
    ("Adorei o seu trabalho! Você é incrível.", "not-toxic"),
    ("Seu namorado é muito feio", "toxic"),
    ("Você é muito gorda", "toxic"),
    ("Você é muito feio", "toxic"),
    ("Vai se ferrar, seu idiota!", "toxic"),
    ("Tenho muito respeito pelo seu esforço.", "not-toxic"),
    ("Você não passa de um imbecil.", "toxic"),
    ("Suas ideias são muito interessantes.", "not-toxic"),
    ("Seu comportamento é inaceitável.", "toxic"),
    ("Obrigado pela sua ajuda, foi fundamental.", "not-toxic"),
    ("Você é um lixo completo.", "toxic"),
    ("Continue assim, estou impressionado.", "not-toxic"),
]

data = []

for text, true_label in data_pairs:
    result = classifier(text)
    data.append({
        "text": text,
        "true_label": true_label,
        "predicted": result[0]["label"],
        "score": result[0]["score"]
    })

df = pd.DataFrame(data)
df
#frases mais curtas sao mais faceis de acertar o label
#as palavras feio/feinho, gorda/gordinha nao aparecem como toxica

,text,true_label,predicted,score
0,Eu nunca vi alguém tão incompetente em toda mi...,toxic,not-toxic,0.793162
1,Deveriam ter escolhido uma modelo menos gordin...,toxic,not-toxic,0.921384
2,"Deveriam ter escolhido uma modelo menos gorda,...",toxic,not-toxic,0.836687
3,"Seu namorado é muito feinho, você não tem verg...",toxic,not-toxic,0.928322
4,"Seu namorado é muito feio, você não tem vergon...",toxic,not-toxic,0.734014
5,"Seu namorado é muito ridículo, você não tem ve...",toxic,toxic,0.657711
6,Parabéns pelo seu trabalho árduo e dedicação. ...,not-toxic,not-toxic,0.978449
7,É inspirador ver o quanto você se esforça para...,not-toxic,not-toxic,0.978444
8,Sua capacidade de resolver problemas de forma ...,not-toxic,not-toxic,0.979144
9,Meus amigos são pessoas muito idiotas por mais...,toxic,not-toxic,0.509544


Referências

MORAITES, A. Fine-tuning RoBERTa for Topic Classification with Hugging Face Transformers and Datasets Library. Disponível em: https://medium.com/@achillesmoraites/fine-tuning-roberta-for-topic-classification-with-hugging-face-transformers-and-datasets-library-c6f8432d0820